<a href="https://colab.research.google.com/github/BRV12G/Final_year_Project/blob/main/classification%2Bregression_Deep_learning_on_fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

# Load your dataset
data = pd.read_csv('/content/updated_cleaned_diet_split_exercises.csv')  # Replace with your dataset path

# Step 1: Drop the ID column
data.drop(columns=['ID'], inplace=True, errors='ignore')

# Step 2: Handle missing values in d_vegetables, d_juice, d_proteinintake using mode
# mode_imputer = SimpleImputer(strategy='most_frequent')
# data['d_vegetables'] = mode_imputer.fit_transform(data[['d_vegetables']])
# data['d_juice'] = mode_imputer.fit_transform(data[['d_juice']])
# data['d_proteinintake'] = mode_imputer.fit_transform(data[['d_proteinintake']])
# Step 2: Handle missing values in d_vegetables, d_juice, d_proteinintake using mode
for col in ['d_vegetables', 'd_juice', 'd_proteinintake']:
    # Ensure all entries are strings; replace non-string values with NaN
    data[col] = data[col].apply(lambda x: str(x) if isinstance(x, str) else np.nan)

# Use mode imputation for missing values
mode_imputer = SimpleImputer(strategy='most_frequent')
data[['d_vegetables', 'd_juice', 'd_proteinintake']] = mode_imputer.fit_transform(
    data[['d_vegetables', 'd_juice', 'd_proteinintake']]
)


# Step 3: Scale numeric columns
scaler = StandardScaler()
numeric_columns = ['Age', 'Height', 'Weight', 'Duration_of_Workout', 'BMI', 'Water_Intake(Litres)' ,'Calories_Burnt']
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Step 4: Calculate BMI
# Assuming Height is in cm, convert it to meters for BMI calculation
data['Height_m'] = data['Height'] / 100
data['BMI'] = data['Weight'] / (data['Height_m'] ** 2)
data.drop(columns=['Height_m'], inplace=True)

# Step 5: Assign BMI class based on BMI values
def assign_bmi_class(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif 18.5 <= bmi < 24.9:
        return 'Normal'
    elif 25 <= bmi < 29.9:
        return 'Overweight'
    else:
        return 'Obese'

data['BMI_Class'] = data['BMI'].apply(assign_bmi_class)

# Encode BMI_Class using LabelEncoder
le_bmi_class = LabelEncoder()
data['BMI_Class'] = le_bmi_class.fit_transform(data['BMI_Class'])

# Step 6: Binary label encoding for gender, hypertension, diabetes
binary_columns = ['Gender', 'Hypertension', 'Diabetes']
label_encoders = {}
for col in binary_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].str.lower())
    label_encoders[col] = le

# Step 7: Handle comma-separated values in d_vegetables, d_juice, d_proteinintake
for col in ['d_vegetables', 'd_juice', 'd_proteinintake']:
    # Split values into lists
    data[col] = data[col].str.lower().str.split(', ')
    # Multi-hot encoding
    unique_values = set(val for sublist in data[col].dropna() for val in sublist)
    for value in unique_values:
        data[f'{col}_{value}'] = data[col].apply(lambda x: 1 if x is not np.nan and value in x else 0)
    # Drop the original column
    data.drop(columns=[col], inplace=True)

# Step 8: One-hot encoding for specified columns
one_hot_columns = ['Fitness_Goal', 'Fitness_Type', 'Exercise1', 'Exercise2', 'Exercise3' , 'Equipment1', 'Equipment2']
ohe = OneHotEncoder(sparse_output=False, drop='first')
encoded_features = ohe.fit_transform(data[one_hot_columns])
encoded_df = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(one_hot_columns))
data = pd.concat([data.reset_index(drop=True), encoded_df], axis=1)
data.drop(columns=one_hot_columns, inplace=True)


# Sanitize column names
data.columns = data.columns.str.replace(r'\s+', '_', regex=True)  # Replace spaces with underscores
data.columns = data.columns.str.replace(r'[^A-Za-z0-9_]', '', regex=True)  # Remove special characters


# Step 9: Handle unknown inputs (replace them with "unknown" or a specific value)
data.replace('unknown', np.nan, inplace=True)
data.fillna('unknown', inplace=True)

# Step 10: Convert the entire dataset to lowercase
for col in data.select_dtypes(include=['object']).columns:
    data[col] = data[col].str.lower()

# Save the preprocessed dataset
data.to_csv('preprocessed_dataset.csv', index=False)

print("Preprocessing complete. The preprocessed dataset is saved as 'preprocessed_dataset.csv'.")

# Display column names in a compact format
print("Columns after preprocessing:", list(data.columns))
















import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Embedding, Flatten, concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score, accuracy_score
import numpy as np

# Assume `data` is preprocessed according to the earlier steps.

# Input Categorical Columns
# categorical_features = ['Gender', 'Hypertension', 'Diabetes', 'BMI_Class', 'Fitness_Goal', 'Fitness_Type']


# Input Numeric Columns
numerical_features = ['Age', 'Height', 'Weight', 'BMI', 'Duration_of_Workout']

# Output Categorical Columns
# output_categorical_features = ['Exercise1', 'Exercise2', 'Exercise3', 'Equipment1', 'Equipment2', 'd_vegetables', 'd_juice', 'd_proteinintake']

# Output Numeric Columns
output_numeric_features = ['Calories_Burnt', 'Water_IntakeLitres']

# Identify One-Hot Encoded Columns
one_hot_encoded_columns = [col for col in data.columns if col.startswith(('Gender_', 'Hypertension_', 'Diabetes_', 'BMI_Class_', 'Fitness_Goal_', 'Fitness_Type_'))]
categorical_features = one_hot_encoded_columns

output_categorical_features = [col for col in data.columns if col.startswith(('Exercise1_', 'Exercise2_', 'Exercise3_', 'Equipment1_', 'Equipment2_', 'd_vegetables_', 'd_juice_', 'd_proteinintake_'))]


# Separate features and targets
X = data[categorical_features + numerical_features]
y_categorical = data[output_categorical_features]
y_numeric = data[output_numeric_features]

# Train-Test Split
X_train, X_temp, y_cat_train, y_cat_temp, y_num_train, y_num_temp = train_test_split(
    X, y_categorical, y_numeric, test_size=0.2, random_state=42
)
X_val, X_test, y_cat_val, y_cat_test, y_num_val, y_num_test = train_test_split(
    X_temp, y_cat_temp, y_num_temp, test_size=0.5, random_state=42
)

# Inputs for the model
numeric_input = Input(shape=(len(numerical_features),), name="numeric_input")
categorical_inputs = []

# Embedding for categorical inputs
embedded_layers = []
for cat_col in categorical_features:
    unique_vals = len(data[cat_col].unique())
    cat_input = Input(shape=(1,), name=f"{cat_col}_input")
    cat_embedding = Embedding(input_dim=unique_vals + 1, output_dim=min(50, (unique_vals // 2) + 1), name=f"{cat_col}_embedding")(cat_input)
    cat_flatten = Flatten()(cat_embedding)
    categorical_inputs.append(cat_input)
    embedded_layers.append(cat_flatten)

# Combine numeric and categorical features
combined_input = concatenate([numeric_input] + embedded_layers)

# Hidden Layers
hidden = Dense(128, activation='relu')(combined_input)
hidden = BatchNormalization()(hidden)
hidden = Dropout(0.3)(hidden)
hidden = Dense(64, activation='relu')(hidden)
hidden = BatchNormalization()(hidden)
hidden = Dropout(0.3)(hidden)

# Output Layers
categorical_outputs = []
for cat_col in output_categorical_features:
    output_layer = Dense(len(data[cat_col].unique()), activation='softmax', name=f"{cat_col}_output")(hidden)
    categorical_outputs.append(output_layer)

numeric_outputs = Dense(len(output_numeric_features), activation='linear', name="numeric_output")(hidden)

# Model
model = Model(inputs=[numeric_input] + categorical_inputs, outputs=categorical_outputs + [numeric_outputs])
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={**{f"{col}_output": "categorical_crossentropy" for col in output_categorical_features}, "numeric_output": "mse"},
    metrics={**{f"{col}_output": "accuracy" for col in output_categorical_features}, "numeric_output": "mae"}
)

# Prepare inputs for training
numeric_data_train = X_train[numerical_features].values
categorical_data_train = [X_train[cat].values for cat in categorical_features]

numeric_data_val = X_val[numerical_features].values
categorical_data_val = [X_val[cat].values for cat in categorical_features]

# Prepare outputs
categorical_output_train = [tf.keras.utils.to_categorical(y_cat_train[col]) for col in output_categorical_features]
categorical_output_val = [tf.keras.utils.to_categorical(y_cat_val[col]) for col in output_categorical_features]

numeric_output_train = y_num_train.values
numeric_output_val = y_num_val.values

# Training the Model
history = model.fit(
    [numeric_data_train] + categorical_data_train,
    categorical_output_train + [numeric_output_train],
    validation_data=(
        [numeric_data_val] + categorical_data_val,
        categorical_output_val + [numeric_output_val]
    ),
    epochs=50,
    batch_size=32
)

# Evaluate the Model
numeric_data_test = X_test[numerical_features].values
categorical_data_test = [X_test[cat].values for cat in categorical_features]

categorical_output_test = [tf.keras.utils.to_categorical(y_cat_test[col]) for col in output_categorical_features]
numeric_output_test = y_num_test.values

evaluation = model.evaluate(
    [numeric_data_test] + categorical_data_test,
    categorical_output_test + [numeric_output_test]
)
print("Evaluation Results:", evaluation)

# Save the entire model (architecture + weights)
model.save('/content/model_name.h5')

mean_accuracy = 0
num_categories = len(output_categorical_features)
# Metrics for Categorical Outputs
for idx, cat_col in enumerate(output_categorical_features):
    y_pred = np.argmax(model.predict([numeric_data_test] + categorical_data_test)[idx], axis=1)
    y_true = np.argmax(categorical_output_test[idx], axis=1)
    print(f"Metrics for {cat_col}:")
    print("Classification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    # Calculating accuracy for this category
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy for {cat_col}: {accuracy:.4f}\n")

    # Accumulating the accuracy for mean calculation
    mean_accuracy += accuracy
    # Calculating mean accuracy across all categorical outputs
mean_accuracy /= num_categories
print(f"Mean Accuracy of the Model: {mean_accuracy:.4f}")

# Metrics for Numeric Outputs
y_pred_numeric = model.predict([numeric_data_test] + categorical_data_test)[-1]
print("Regression Metrics:")
print("MSE:", mean_squared_error(numeric_output_test, y_pred_numeric))
print("R² Score:", r2_score(numeric_output_test, y_pred_numeric))



Preprocessing complete. The preprocessed dataset is saved as 'preprocessed_dataset.csv'.
Columns after preprocessing: ['Gender', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI', 'BMI_Class', 'Duration_of_Workout', 'Calories_Burnt', 'Water_IntakeLitres', 'd_vegetables_carrots', 'd_vegetables_capers', 'd_vegetables_mushroom', 'd_vegetables_mushroon', 'd_vegetables_mixed_greens', 'd_vegetables_broccoli', 'd_vegetables_and_iceberg_lettuce', 'd_vegetables_cherry_tomatoes', 'd_vegetables_sweet_potato', 'd_vegetables_garlic', 'd_vegetables_lettuce', 'd_vegetables_icebetg_lettuce', 'd_vegetables_leafy_greens', 'd_vegetables_green_papper_and_water_chestnut', 'd_vegetables_bell_peppers', 'd_vegetables_and_bell_peppers', 'd_vegetables_green_papper', 'd_vegetables_tomatoes', 'd_vegetables_onion', 'd_vegetables_and_lettuce', 'd_vegetables_capers_and_iceberg_lettuce', 'd_vegetables_celery', 'd_vegetables_cucumbers', 'd_vegetables_roma_tomatoes', 'd_vegetables_spinach', 'd_juice_and_wate

Evaluation Results: [37.702842712402344, 0.6401361227035522, 0.373913437128067, 0.6386793851852417, 0.39978229999542236, 0.2470826506614685, 0.5121011734008789, 0.37391528487205505, 0.24708370864391327, 0.22948461771011353, 0.595906674861908, 0.4733722507953644, 0.6386688947677612, 0.20997658371925354, 0.24671106040477753, 0.24708163738250732, 0.20998956263065338, 0.6886327862739563, 0.20998625457286835, 0.424345999956131, 0.13944247364997864, 0.2236778885126114, 0.2470831423997879, 0.24708323180675507, 0.4737686812877655, 0.4243631064891815, 0.4243408143520355, 0.22948165237903595, 0.3739157021045685, 0.6861572861671448, 0.4243551194667816, 0.5201089978218079, 0.4243491590023041, 0.5200914144515991, 0.37391236424446106, 0.3506599962711334, 0.22947707772254944, 0.2294846773147583, 0.42432475090026855, 0.24960893392562866, 0.3552515506744385, 0.42433685064315796, 0.424348920583725, 0.3739175498485565, 0.6386531591415405, 0.47376927733421326, 0.22367402911186218, 0.3739131987094879, 0.22

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Metrics for d_vegetables_capers:
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.93      1279
           1       0.00      0.00      0.00       180

    accuracy                           0.88      1459
   macro avg       0.44      0.50      0.47      1459
weighted avg       0.77      0.88      0.82      1459

Confusion Matrix:
 [[1279    0]
 [ 180    0]]
Accuracy for d_vegetables_capers: 0.8766

10/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Metrics for Equipment2_Medicine_Ball:
Classification Report:
               precision    recall  f1-score   support

           0       0.83      1.00      0.91      1211
           1       0.00      0.00      0.00       248

    accuracy                           0.83      1459
   macro avg       0.42      0.50      0.45      1459
weighted avg       0.69      0.83      0.75      1459

Confusion Matrix:
 [[1211    0]
 [ 248    0]]
Accuracy for Equipment2_Medicine_Ball: 0.8300

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Metrics for Equipment2_Treadmill:
Classification Report:
               precision    recall  f1-score   support

           0       0.83      1.00      0.91      1211
           1       0.00      0.00      0.00       248

    accuracy                           0.83      1459
   macro avg       0.42      0.50      0.45      1459
weighted avg       0.69      0.83      0.75      1459

Confusion Matrix:
 [[1211    0]
 [ 248    0]]
Accuracy for Equipment2_Treadmill: 0.8300

Mean Accuracy of the Model: 0.8472
11/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Regression Metrics:
MSE: 1.0101613186673495
R² Score: -0.0024436818859071563
